In [39]:
import panel as pn
from panel_vega_view import VegaView

pn.extension('ace', 'jsoneditor', 'vega')


spec = {
  "$schema": "https://vega.github.io/schema/vega-lite/v5.json",
  "description": "Drag out a rectangular brush to highlight points.",
  "data": {"url": "https://raw.githubusercontent.com/vega/vega/refs/heads/main/docs/data/cars.json"},
  "params": [{
    "name": "brush",
    "select": "interval",
    "value": {"x": [55, 160], "y": [13, 37]}
  },{
    "name": "point_sel",
    "select": "point",
  },
  ],
  "mark": "point",
  "encoding": {
    "x": {"field": "Horsepower", "type": "quantitative"},
    "y": {"field": "Miles_per_Gallon", "type": "quantitative"},
    "color": {
      "condition": {"param": "brush", "field": "Cylinders", "type": "ordinal"},
      "value": "grey"
    },
    "size": {
      "condition": {"param": "point_sel", "value": 100},
      "value": 10,
    },
    "facet": {
      "field": "Cylinders",
      "columns": 2
    }
  }
}


spec = {
  "$schema": "https://vega.github.io/schema/vega-lite/v5.json",
  "data": {"url": "https://raw.githubusercontent.com/vega/vega/master/docs/data/movies.json"},
  "transform": [{
    "filter": {"and": [
      {"field": "IMDB Rating", "valid": True},
      {"field": "Rotten Tomatoes Rating", "valid": True}
    ]}
  }],
  "params": [
      {
        "name": "colorDomain",
        "value": [0, 30],
      },
      {
        "name": "brush",
        "select": "interval",
        # "value": {"x": [55, 160], "y": [13, 37]}
      },
      {
          "name": "x_bins",
          "value": 40,
      },
      {
          "name": "y_bins",
          "value": 40,
      },
      {
        "name": "point_sel",
        "select": "point",
      },
  ],
  "mark": {
      "type": "rect",
      "width": 5,
      "height": 5,
  },
  # "width": "container",
  "height": 300,
  "width": 300,
  "encoding": {
    # "size": 5,
    "x": {
      "bin": {"maxbins": 40},
      "field": "IMDB Rating",
      "type": "quantitative"
    },
    "y": {
      "bin": {"maxbins": 40},
      "field": "Rotten Tomatoes Rating",
      "type": "quantitative"
    },
    "color": {
      "condition": {
        "param": "brush",
        "aggregate": "count",
        "type": "quantitative",
        "scale": {
            "domain": {"expr": "colorDomain"},
        }
      },
      "value": "grey"
    }
  },
  "config": {
    "view": {
      "stroke": "transparent"
  }
    }
}


vv = VegaView(
    spec=spec,
    opt={
        "actions": True,
    },
    signal_names=[
        "point_sel",
        "brush",
        "colorDomain",
        # "height",
        # "brush_x",
        # "brush_tuple",
    ],
)

json_editor = pn.widgets.JSONEditor(value=vv.param.signals, width=300)
json_editor.jslink(vv, code={"value": """
console.log('JSON', source);
console.log('JSON', target.model_proxy.uuid);
let vegaView = window.vegaViews[target.model_proxy.uuid];

Object.entries(source.value).forEach(([key, value]) => {
  console.log(key, value);
  if (value != null && value != undefined)
    vegaView.signal(key, value);
});
vegaView.runAsync();
"""})
# json_editor2 = pn.widgets.JSONEditor(value=vv.param.data, width=300)
color_lims = pn.widgets.RangeSlider(name='Color limits', start=0, end=125, value=(0, 40), step=1)
color_lims.jslink(vv, code={'value': """
let vegaView = window.vegaViews[target.model_proxy.uuid];

vegaView.signal("colorDomain", source.value);
vegaView.runAsync();
"""})

width_slider = pn.widgets.IntSlider(name='Width', start=100, end=1000, value=300)
width_slider.jslink(vv, code={'value': """
              
let signals = {...target.model_proxy.signals};
signals["width"] = source.value;
target.model_proxy.signals = signals;
"""})

maxbins_slider = pn.widgets.IntSlider(name='MaxBins', start=1, end=100, value=5)
maxbins_slider.jslink(vv, code={'value': """       
let spec = {...target.model_proxy.spec};
console.log('SPEC', spec);
spec.encoding.x.bin.maxbins = source.value;
spec.encoding.y.bin.maxbins = source.value;
target.model_proxy.spec = spec;
"""})

uuid = pn.widgets.TextInput(value=vv.param.uuid)

pn.Column(
  pn.Row(
      vv,
      json_editor,
      # json_editor2,
  ),
  color_lims,
  # width_slider,
  maxbins_slider,
  uuid,
)

BokehModel(combine_events=True, render_bundle={'docs_json': {'348abeaf-d5c9-4cc1-a6de-a412ef55d5f8': {'version…

In [57]:
from copy import deepcopy

new_spec = deepcopy(vv.spec)
new_spec['mark'] = 'rect'
new_spec['encoding']['x']['bin']['maxbins'] = 20
new_spec['encoding']['y']['bin']['maxbins'] = 20
vv.spec = new_spec